# KLayout Scripting Guide

A guide to scripting in KLayout using the Python Library. This guide is inspired by the [PyPi](https://www.klayout.org/klayout-pypi) API. This guide will show you how to generate cells (structures) in KLayout and build nested cells (using a pattern in another pattern).

KLayout allows users to build patterns in a database (GDSII) file, which can be read by the Raith Voyager system. Patterns may be placed in nested cells, which is the equivalent of building a position list, but can also be seen individually within the database allowing users to build a position list using the Raith tool.

## Contents

* [Imports](#imports)
* [Organisation](#organisation)
* [Flat Gratings](#flat-gratings)
    * [Grating Parameters](#grating-parameters)
    * [Database Setup](#database-setup)
    * [Grating Layer](#grating-layer)
    * [Write Out](#write-out)
    * [Results](#result)
* [Hierarchical Gratings](#hierarchical-gratings)
    * [Database Setup](#hierarchical-database-setup)
    * [Grating Cell](#grating-cell)
    * [Results](#hierarchical-result)
    * [Nested Grating Bars](#nested-grating-bars)
    * [Results](#nested-grating-result)
    * [Write Out](#hierarchical-write-out)
* [Rotations and Transformations](#rotations-and-transformations)
    * [Database Setup](#rotations-database-setup)
    * [Grating Cell](#rotations-grating-cell)
    * [Rotating and Transforming](#rotating-and-transforming-cells)
    * [Results](#rotation-grating-results)
    * [Write Out](#rotation-write-out)

## Imports

As is convention when building a python script, we will start by importing the key python libraries required to use the code in this document. The key Python library required is klayout.db.

Remember that Python libraries can be installed using the pip tool, or sudo apt-get on linux devices:

* python -m pip install klayout
* sudo apt-get install python-klayout

Other key libraries required are math, to perform calculations, numpy and pathlib for universal directory paths.

In [121]:
import math
import klayout.db as db

from pathlib import Path

## Organisation

We will begin by setting up the organisation (directory tree). This directory contains the scripts required to complete the processing and save out to a directory called "Data". This should not need changing, but if you would rather save the database files elsewhere, please adjust the out_path as required.

As we are using pathlib to set our directory paths, this is a simple f-string in python.

[Back To Top](#contents)

In [122]:
root = Path().absolute()
out_path = Path(f'{root}/Data')
print(out_path)

c:\Users\joshs\github\Cleanroom_Toolkit\GDSII_KLayout\Data


## Flat Gratings

Flat gratings are one-dimensional periodic structures with a constant period and fill factor. The most basic of periodic structures. We will begin by setting up the grating parameters and then we will create the individual grating cell and nest with labelling text.

### Grating Parameters

We begin by setting the initial grating parameters, namely the height and width of the overall grating, the grating period, filling factor (i.e., how much of the period is filled with the grating material), and the dose factor (the dose scaling factor used by the Raith system).

In KLayout, the dose factor is scaled differently to the Raith system. The Raith system may use a dose factor of "1", and in KLayout this would be "1000", and "1.2" is "1200" etc. This is a simple scaling factor that we will add later, for now please use the Raith dose factor that you would like to achieve, e.g., "1".

Please note that KLayout defaults to millimeter scaling, but later we will set the database units to micrometers. Hence the following parameters are set to micrometers.

[Back To Top](#contents)

In [123]:
grating_height = 500
grating_width = 500
dose_factor = 1
grating_period = 0.550
fill_factor = 0.7

### Database Setup

Now we will set up the KLayout database to contain our grating structures. We do this using the db.Layout() function, and then set the database units to be micrometers.

As KLayout is a database of cells, we need to give the database a top cell, this can be called whatever you would like, but the convention is to use "TOP". Please change the naming however you would like it to appear.

[Back To Top](#contents)

In [124]:
layout = db.Layout()
layout.dbu = 0.001  # millimeters to micrometers
top_cell = layout.create_cell("TOP")
top_layer = layout.layer(1, dose_factor * 1000)

### Grating Layer

Now that we have a cell called "TOP", we want to start building our grating. We start by finding the number of periods in the grating width as a whole number, then we create a database point (the bottom left point of the grating). From this point we extend the box as a vector of the width and height of the grating bar.

Finally, we add the shape to the grating layer and repeat for the number of boxes in the total grating width. We are just going to add our grating to the top cell directly, later on we will discuss nesting cells.

Recall that the filling factor is the area of the period filled by the grating and that writing into positive photoresist is writing the air, so we need to introduce a (1-fill_factor) into the calculation.

You will see that we start the grating point at a number (i) multiplied by the grating period, and then extend this point to create a box using the database vector.

[Back To Top](#contents)

In [125]:
number_of_boxes = math.floor(grating_width / grating_period)
for i in range(0, number_of_boxes):
    starting_point = db.DPoint(i * grating_period, 0)
    grating_box = db.DBox(
        starting_point,
        (
            starting_point +
            db.DVector(grating_period * (1 - fill_factor), grating_height)
        )
    )
    top_cell.shapes(top_layer).insert(grating_box)

### Write Out

Now we have built the grating structure and we want to write it out and save as a database file. For the purpose of this guide, I will use the file name "Example_FlatGratings.gds", and all this should contain is a cell called "TOP" with a grating of 550nm period in a 500x500 micrometer square. The grating should have a fill factor of 0.7, which will be the white area in KLayout, the written boxes (pink/purple) in colour, will be 0.3 fill factor.

[Back To Top](#contents)

In [126]:
layout.write(f'{out_path}/Example_FlatGratings.gds')

### Result

The result should be something that looks like this:

![Example_FlatGratings.png](./src/Images/Example_FlatGratings.png)

A flat grating of period 550nm with a total grating size of 500x500um, and in a cell called "TOP".

[Back To Top](#contents)

## Hierarchical Gratings

Now we will discuss nesting cells inside another cell. You may be familiar with the Hierarchy system, or nested system, or layered system. Essentially we are going to build a basic structure in a cell as before, but then we are going to nest that cell within a higher cell, and repeat the structure to create a grating.

Again, we will only consider flat gratings in this section.

### Hierarchical Database Setup

Now we will set up the KLayout database to contain our grating structures. We do this in the same way as before with db.Layout() and we create a top cell, named "TOP". Here we have not defined a layer or dose, as this will be done within the grating cell. Note that we have set the database units to um.

[Back To Top](#contents)

In [127]:
layout = db.Layout()
layout.dbu = 0.001
top_cell = layout.create_cell("TOP")

### Grating Cell

Now that we have a top cell and a database, we can begin by setting up the grating cell, before inserting it into the top cell.

We create a new cell, specifically for the grating, named "Grating" and apply a layer to that cell with the dose factor and dose factor scaling of 1000 as before. Then we create the grating, in much the same was as in [Grating Layer](#grating-layer) but we apply the grating to the grating cell.

Once we have got the grating, we add the grating cell to the top cell using the .insert() function and giving it a translational vector. Because we began the grating at (0, 0) in the grating cell, we are moving the (0, 0) position using this grating. For now, we will not move the grating in the top cell, hence the vector is given a (0, 0).

[Back To Top](#contents)

In [128]:
grating_cell = layout.create_cell("Grating")
grating_layer = layout.layer(1, dose_factor * 1000)
number_of_boxes = math.floor(grating_width / grating_period)
for i in range(0, number_of_boxes):
    starting_point = db.DPoint(i * grating_period, 0)
    grating_box = db.DBox(
        starting_point,
        (
            starting_point +
            db.DVector(grating_period * (1 - fill_factor), grating_height)
        )
    )
    grating_cell.shapes(grating_layer).insert(grating_box)
top_cell.insert(
    db.DCellInstArray(
        grating_cell.cell_index(),
        db.DTrans(
            db.DVector(0, 0)
        )
    )
)

cell_index=1 r0 0,0

### Hierarchical Result

The result should be something that looks like this:

![Example_FlatGratings.png](./src/Images/Example_HierarchicalGratings.png)

A flat grating of period 550nm with a total grating size of 500x500um, and in a cell called "Grating" which is nested in the "TOP" cell.

### Nested Grating Bars

Taking this a step further, what if we build the grating bar and nest it within the grating cell which is further nested in the top cell?

We start by building a bar_cell and giving it its own dose, this is purely for appearance in the example database as the two will appear as different colours. We build the bar by giving it a starting point, where here we have arbitrarily given it a (0, 0) starting point, and then we give it a vector as the grating period multiplied by (1 - fill factor), and the grating height. Then we add the bar shape to the bar cell, in much the same fashion as before. Then we use the same formulation to add the bar cell to the grating cell in a periodic repetition, finally adding the grating cell to the top cell as before.

Here I have added the second grating to the top cell at a 600um spacing to the first grating so that both are visible in the top cell. This is the same method that will be used to create rows of gratings going forward.

[Back To Top](#contents)

In [129]:
bar_cell = layout.create_cell("Bar")
bar_layer = layout.layer(2, dose_factor * 1000)
starting_point = db.DPoint(0, 0)
grating_bar = db.DBox(
    starting_point,
    (
        starting_point +
        db.DVector(grating_period * (1 - fill_factor), grating_height)
    )
)
bar_cell.shapes(bar_layer).insert(grating_bar)
grating_cell2 = layout.create_cell("Grating_2")
x_vector = db.DVector(grating_period, 0)
number_of_boxes = math.floor(grating_width / grating_period)
for i in range(0, number_of_boxes):
    grating_cell2.insert(
        db.DCellInstArray(
            bar_cell.cell_index(),
            db.DTrans(
                i * grating_period,
                0
            )
        )
    )
top_cell.insert(db.DCellInstArray(grating_cell2.cell_index(), db.DTrans(db.DVector(600, 0))))

cell_index=3 r0 600000,0

### Nested Grating Result

Now we see two gratings, spaced by 600um, where grating 2 is a nested cell or bars repeated to make the grating. While this is not immensely useful as most people will want to create a grating in one cell, it can be useful for other design files.

![Example_FlatGratings.png](./src/Images/Example_HierarchicalGratings2.png)

### Hierarchical Write Out

Now we have built the grating cell and applied it to the top cell, we want to write out. This is much the same as before, but I will call this "Example_HierarchicalGratings.gds". This should create a cell called "TOP" under which there is a cell called "Grating" into which we wrote our grating pattern, with period, fill factor, and dose as detailed above.

[Back To Top](#contents)

In [130]:
layout.write(f'{out_path}/Example_HierarchicalGratings.gds')

## Rotations and Transformations

What if we want to change the orientation of a grating, so that instead of the bars running vertically as they have done in previous examples, they now run horizontally, or at any angle?

Again, we will only consider flat gratings in this section.

### Rotations Database Setup

Now we will set up the KLayout database to contain our grating structures. We do this in the same way as before, creating a top cell, named "TOP", and defining the database units as um.

[Back To Top](#contents)

In [131]:
layout = db.Layout()
layout.dbu = 0.001
top_cell = layout.create_cell("TOP")

### Rotations Grating Cell

Now that we have a top cell and a database, we can begin by setting up the grating cell, before inserting it into the top cell.

Here we will use only one grating cell, and we will then apply four copies of this to the top cell to demonstrate the complex transformations. We have used the bar cell and period inserts to create a nested grating cell. Note that here we have not inserted the grating cell into the top cell.

[Back To Top](#contents)

In [132]:
bar_cell = layout.create_cell("Bar")
bar_layer = layout.layer(1, dose_factor * 1000)
starting_point = db.DPoint(0, 0)
grating_bar = db.DBox(
    starting_point,
    (
        starting_point +
        db.DVector(grating_period * (1 - fill_factor), grating_height)
    )
)
bar_cell.shapes(bar_layer).insert(grating_bar)
grating_cell = layout.create_cell("Grating")
x_vector = db.DVector(grating_period, 0)
number_of_boxes = math.floor(grating_width / grating_period)
for i in range(0, number_of_boxes):
    grating_cell.insert(
        db.DCellInstArray(
            bar_cell.cell_index(),
            db.DTrans(
                i * grating_period,
                0
            )
        )
    )

### Rotating and Transforming Cells

To rotate and transform a cell, we use the db.ICplxTrans() function which takes the arguments (scale, rotation, mirror, x, y). Here we have used clear variable names to indicate their purpose. Once complete, we then insert the grating cell into the top cell as before, and supply the transformations or rotations in the DCellInstArray() function.

[Back To Top](#contents)

In [133]:
rotation_angle_1 = 90
rotation_angle_2 = 45
scale_factor_1 = 1
scale_factor_2 = 2
mirror = False
transform_1 = db.ICplxTrans(scale_factor_1, 0, mirror, 0, 0)
transform_2 = db.ICplxTrans(scale_factor_1, rotation_angle_1, mirror, 1100, 0)
transform_3 = db.ICplxTrans(scale_factor_1, rotation_angle_2, mirror, 1600, 0)
transform_4 = db.ICplxTrans(scale_factor_2, 0, mirror, 2200, 0)
top_cell.insert(db.DCellInstArray(grating_cell.cell_index(), transform_1))
top_cell.insert(db.DCellInstArray(grating_cell.cell_index(), transform_2))
top_cell.insert(db.DCellInstArray(grating_cell.cell_index(), transform_3))
top_cell.insert(db.DCellInstArray(grating_cell.cell_index(), transform_4))

cell_index=2 r0 *2 2200000,0

### Rotation Grating Results

Now we should see four gratings, one as the original, one rotated by 90 degrees (note the difference in coordinates from the rotation), one rotated by 45 degrees, and one scaled by a factor 2 (note that this increases all dimensions by a factor 2, so the bar width and period have also scaled by a factor 2).

![Example_TransformationGratings.png](./src/Images/Example_TransformationGratings.png)

### Rotation Write Out

Now we have built the grating cell and applied it to the top cell with the rotations and transformations, we want to write out. This is done exactly as before, but I will call this "Example_TransformationGratings.gds". This should create a cell called "TOP" under which there is a cell called "Grating" into which we wrote our grating pattern, with period, fill factor, and dose as detailed above.

[Back To Top](#contents)

In [134]:
layout.write(f'{out_path}/Example_TransformationGratings.gds')